In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import pyEDM
import seaborn as sns
from joblib import Parallel, delayed
from numpy.random import default_rng
from scipy.spatial.distance import squareform, pdist
from sklearn.preprocessing import StandardScaler



def recurrence_matrix(x_e, quantile_per=12.5):
    """
    Generates binary recurrence matrix, where the closest ``quantile_per``% of pairwise distances are assigned a 1,
    and all other pairs assigned a 0.
    :param x_e: embedded dataframe of the required form such as that generated through ``pyEDM.Embed()``
    :param quantile_per: percentile of distances to set equal to 1
    :return: dataframe of recurrence matrix
    """
    dist_mat = squareform(pdist(x_e.values, metric="chebyshev"))
    binary_dist_mtx = (dist_mat <= np.percentile(dist_mat, quantile_per)).astype(int)
    # print("1s: ", np.count_nonzero(binary_dist_mtx) / (len(binary_dist_mtx) ** 2))
    return binary_dist_mtx

def twins_list(len_df, Ng, r_mtx, obs_per_year=12):
    """
    Creates an array of all twin points. Twin points are defined as those which are sufficiently close together in
    state space (same column in the recurrence matrix) and have the same seasonality (measurements from the same month)
    :param len_df: length of original dataframe (i.e. length of time series)
    :param Ng: length of embedded dataframe
    :param r_mtx: recurrence matrix dataframe
    :param obs_per_year: seasonality of the data
    :return: array of twins, formatted in two different ways. A_arr is the useful format for what follows
    """
    Eminus1 = len_df - Ng
    Alist = [[i] for i in range(len_df)]
    for i, j in combinations(range(Ng), 2):  # does not include repeats
        if np.array_equal(r_mtx[:, i], r_mtx[:, j]) and (j - i) % obs_per_year == 0:
            Alist[i + Eminus1].append(j + Eminus1)
            Alist[j + Eminus1].append(i + Eminus1)
    Q = np.array([len(twins) for twins in Alist])
    A_arr = (
        np.zeros((len_df, np.max(Q)), dtype=int) - 1000
    )  # -1000 here after trying it
    for i in range(len_df):
        A_arr[i, : Q[i]] = Alist[i]
    return A_arr, Q



def network_surrogates(len_df, twins_arr, Q, Ng, M=100, obs_per_year=12):
    """
    Generate all surrogates using a network method described in report. First generate points in phase space,
    and then append on initial time series
    :param len_df: length of original dataframe (i.e. length of time series)
    :param twins_arr: array of the twin points of every point (e.g. ouput of twins list
    :param Q: array of degrees of each node
    :param Ng: length of embedded dataframe/ number of nodes
    :param M: number of walkers starting on each node
    :param obs_per_year: sampling observations per year of data
    :return: arrays for which each column is a surrogate
    """
    nodes = np.arange(Ng - 2)
    Eminus1 = len_df - Ng
    start_nodes = nodes[nodes % obs_per_year == 0]
    start_nodes = start_nodes[start_nodes >= obs_per_year]
    num_nodes = len(start_nodes)
    X = np.zeros((Ng, M, num_nodes), dtype=int)
    X[0, :, :] = np.tile(start_nodes, (M, 1))

    rng = default_rng()  # generate random numbers (using numpy generator method)
    rand1 = rng.random((Ng, M, num_nodes))

    for i in range(1, Ng):
        X[i, :, :] = X[i - 1, :, :]
        QX = Q[X[i, :, :]]  # degrees of nodes where each walker is currently
        Ri = (
            rand1[i, :, :] * QX
        )  # Generate array of random numbers, each less than QX[i,j]
        Ri = Ri.astype(int)  # integer conversion
        mask = X[i, :, :] < (Ng - 1)  # relax this condition to let points jump off
        X[i, :, :][mask] = twins_arr[X[i, :, :], Ri][mask]  # update walker locations
        X[i, :, :][X[i, :, :] < (Ng - 1)] += 1
    surrogates = X[:, X[-1, :, :] < (Ng - 1)]
    # back propagate initial times to get to full length of time series
    initial_times = np.add(
        np.tile(surrogates[0, :] - Eminus1, (Eminus1, 1)),
        np.mgrid[:Eminus1, : len(surrogates[0, :])][0],
    )
    return np.concatenate((initial_times, surrogates))  # each column is a surrogate!!


def run_surrogates(df, mae_dict, sp, n_surrogates=5, obs_per_year=12, thresholds=-1):
    """
    Generates all the surrogates using the rEDM method. Note the ability to insert a list of
    thresholds to try. Default is the rEDM implementation list of thresholds to try.
    :param df: input dataframe
    :param mae_dict: dictionary of lists; generate with :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param sp: species for which to generate surrogates
    :param n_surrogates: number of surrogates to generate
    :param obs_per_year: period of seasonality of data
    :param thresholds: list of thresholds to try. Numbers between 5 and 20 recommended.
    :return: embedded dataframe, surrogates, optimal embedding dimension
    """
    surrogate_slice = None
    optE = 0
    len_df = len(df)
    for count in range(25):
        optE = mae_dict[sp][count][0]
        x_e = pyEDM.Embed(dataFrame=df, E=optE, columns=sp)
        Ng = len(x_e)
        if thresholds == -1:
            thresholds = [
                12.5,
                12,
                11,
                10,
                9,
                8,
                7,
                6,
                5,
                15,
                16,
                17,
                18,
                19,
                20,
                4,
            ]
        twins_arr = None
        Q = None
        for thres in thresholds:
            r_mtx = recurrence_matrix(x_e, thres)
            twins_arr, Q = twins_list(
                len_df, Ng=Ng, r_mtx=r_mtx, obs_per_year=obs_per_year
            )
            # if there are at least 10 twins
            if np.sum(Q) - Ng > 10:
                break
        surrogate_slice = network_surrogates(
            len_df, twins_arr, Q, Ng, M=100 * n_surrogates, obs_per_year=obs_per_year
        )
        if len(surrogate_slice[0, :]) >= n_surrogates:
            print("E = {}".format(optE))
            print("Threshold = {}".format(thres))
            break
    rng = default_rng()
    return (
        x_e,
        rng.choice(a=surrogate_slice, size=n_surrogates, replace=False, axis=1),
        optE,
    )


def generate_all_surrogates(df, mae_dict, n_surrogates=5, obs_per_year=12):
    """
    Generates surrogates for all species of a given dataframe, using the :meth:`src.PLTS.networkccm3.run_surrogates`
    method.
    :param df: input dataframe
    :param mae_dict: dictionary of lists; generate with :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param n_surrogates: number of surrogates to use with phase-lock twin surrogate method
    :param obs_per_year: periodicity of time series data (12 for Guadalquivir data, 24 for Maizuru data)
    :return: length of embedded dataframe, array of surrogate time series, array of optimal embedding dimensions
    """
    species_list = list(df.columns[1:])
    len_df = len(df)
    surr_array = np.zeros((len(species_list), len_df, n_surrogates))
    Es = np.zeros(len(species_list))
    for i, sp in enumerate(species_list):
        x_e, surr_array[i], Es[i] = run_surrogates(
            df, mae_dict, sp, n_surrogates=n_surrogates, obs_per_year=obs_per_year
        )
    return x_e, surr_array, Es.astype(int)



def mae_dictionary(df, max_dim=24):
    """
    Creates dictionary with species as keys and each corresponding index a list of embedding dimensions from 1 to
    max_dim in order of optimality according to mean absolute error. See section 3.2 of report.
    :param df: input dataframe of usual form (e.g. that given by :meth:`src.abundance_tools.initialise_dataset`)
    :param max_dim: maximum embedding dimension to include in dictionary. Default is 24, copying Ushio paper
    :return: dictionary of optimal embedding dimension information
    """
    mae_dict = {}
    for _, sp in enumerate(df.columns[1:]):
        MAEs = [[] for _ in range(2, max_dim + 1)]
        for E in range(2, max_dim + 1):
            library_string = "1 {}".format(len(df) - E)
            preds = pyEDM.Simplex(
                dataFrame=df,
                columns=sp,
                target=sp,
                E=E,
                Tp=1,
                lib=library_string,
                pred=library_string,
                exclusionRadius=1,
            )
            MAEs[E - 2] = [
                E,
                np.nanmean(
                    np.abs((preds["Predictions"] - preds["Observations"]).values)
                ),
            ]
        mae_dict[sp] = MAEs
    for sp in mae_dict:
        mae_dict[sp].sort(key=takeSecond)
    return mae_dict

def takeSecond(elem):
    """
    Returns second element of a list. Auxiliary function for :meth:`src.processing.embedding_dimension.mae_dictionary`
    :param elem: list
    :return: second element
    """
    return elem[1]


def twin_surrogate_esn(X, Y, Nsurr, period):
    df = pd.DataFrame()
    df['X'] = X
    df['Y'] = Y
    df.insert(loc=0, column="Time", value=range(len(df)))
    
    df_mae_dict = mae_dictionary(df, max_dim=24)
    _, surr_array, _ = generate_all_surrogates(df, df_mae_dict, n_surrogates = Nsurr, obs_per_year = period)
    
    return surr_array[0,:,:].T, surr_array[1,:,:].T

def DeepESN_real(X, Y, n_data = 50, period = 1, verbose = False):
    
    # Get length of time series
    N = X.shape[0]
    
    # compute surrogates
    Xsurr, Ysurr = twin_surrogate_esn(X, Y, Nsurr = n_data, period = period)
    
    
    # run the deepESN of the surrogates
    xmapy_surr, ymapx_surr = compute_confidence(Xsurr, Ysurr, verbose)

    # run deepESN for actual data
    xmapy, ymapx = compute_lags(X, Y, n_data, verbose)

    return xmapy_surr, ymapx_surr, xmapy, ymapx

In [2]:
env = pd.read_csv('GEnv_NOgaps.csv')
spec = pd.read_csv('GuadaAB.csv')

In [3]:
sig_species = [
    "Mesopodopsis.slabberi",
    "Neomysis.integer",
    "Rhopalophthalmus.tartessicus",
    "Pomatoschistus",
    "Engraulis.encrasicolus",
    "Crangon.crangon",
    "Palaemon.longirostris",
    "Sardina.pilchardus",
    "Palaemon.macrodactylus",
    "Dicentrarchus.punctatus",
    "Liza.ramada",
    "Liza.aurata",
    "Argyrosomus.regius",
    "Chelon.labrosus",
    "Cyprinus.carpio",
]

In [4]:
def initialise_allsp(spec, AB="A", scale=True):
    """
    Initialise dataframe combining time series data from all site and tide pairs at each date.
    :param AB: 'A' for abundance, 'B' for biomass
    :param scale: if True, all time series scaled to 0 mean and unit variance
    :return: pandas dataframe
    """
    spec_df = spec
    spec_df = spec_df[spec_df["Date"] < "2018-07-11"]  # this is key
    sig_species = [
        "Mesopodopsis.slabberi",
        "Neomysis.integer",
        "Rhopalophthalmus.tartessicus",
        "Pomatoschistus",
        "Engraulis.encrasicolus",
        "Crangon.crangon",
        "Palaemon.longirostris",
        "Sardina.pilchardus",
        "Palaemon.macrodactylus",
        "Dicentrarchus.punctatus",
        "Liza.ramada",
        "Liza.aurata",
        "Argyrosomus.regius",
        "Chelon.labrosus",
        "Cyprinus.carpio",
    ]
    dropped = ["A", "B"]
    dropped.remove(AB)
    sig_df = spec_df.drop([dropped[0], "Date", "Code", "Sample"], axis=1)
    pd.to_datetime(sig_df["Date5.1"])
    sig_df = sig_df[sig_df.Specie.isin(sig_species)]
    sig_df = sig_df.loc[:, ~sig_df.columns.str.contains("^Unnamed")]
    grouped_df = sig_df.groupby(["Date5.1", "Specie"], as_index=False).mean()
    grouped_df = grouped_df.drop(["Site", "Tide"], axis=1)
    grouped_df = grouped_df.set_index("Date5.1")
    df = grouped_df.pivot_table(values=AB, index=["Date5.1"], columns=["Specie"])
    df.fillna(method="ffill")  # fill gaps forwards
    if scale:
        scaler = StandardScaler()
        df[df.columns] = scaler.fit_transform(df[df.columns])
    df.insert(loc=0, column="Time", value=range(len(df)))
    df.drop(df.tail(1).index, inplace=True)  # drop last n rows
    return df

In [5]:
def combined_df(env, spec):
    """
    Combined dataframe of environmental and species abundance data, restricted to only include data points for which
    all information is known
    :return: pandas dataframe
    """
    env_df = env
    spec_df = spec

    env_df = env_df[["Code", "TurbidityNTU", "Temp", "Si", "V30", "P30"]]
    env_df.dropna(inplace=True)
    spec_df = spec_df[["Code", "Specie", "A", "B", "Date5.1"]]
    df = pd.merge(spec_df, env_df, how="inner", on="Code")
    df = df[df.Specie.isin(sig_species)]
    df = df[df["Date5.1"] < "2018-07-11"]
    df.drop("B", axis=1, inplace=True)
    comb_df = df.pivot_table(
        values="A", index=["Date5.1", "Code"], columns=["Specie"], fill_value=0
    )
    comb_df.reset_index(level=["Date5.1", "Code"], inplace=True)
    comb_df = pd.merge(comb_df, env_df, how="inner", on="Code")
    comb_df = comb_df.groupby(["Date5.1"], as_index=False).mean()

    scaler = StandardScaler()
    comb_df[sig_species] = scaler.fit_transform(comb_df[sig_species])
    comb_df.insert(loc=0, column="Time", value=range(len(comb_df)))

    comb_df.drop("Code", axis=1, inplace=True)
    comb_df.drop("Date5.1", axis=1, inplace=True)
    return comb_df

In [6]:
def combined_df_ST(env, spec, site=5, tide_list=(1, 3)):
    """
    Same as :meth:`src.environmental.env_ccm.combined_df`. but for a specific site-tide pair.
    :param site: site from which to take data points from
    :param tide_list: Either (1,3) for flood tides or (2,4) for ebb tides
    :return: combined pandas dataframe
    """
    env_df = env
    spec_df = spec

    env_df = env_df[["Code", "TurbidityNTU", "Temp", "Si", "V30", "P30"]]
    env_df.dropna(inplace=True)
    spec_df = spec_df[["Code", "Specie", "A", "B", "Date5.1", "Site", "Tide"]]
    red_spec_df = spec_df[spec_df.Site.eq(site) & spec_df.Tide.isin(tide_list)]
    df = pd.merge(red_spec_df, env_df, how="inner", on="Code")
    df = df[df.Specie.isin(sig_species)]
    df = df[df["Date5.1"] < "2018-07-11"]
    df.drop("B", axis=1, inplace=True)
    comb_df = df.pivot_table(
        values="A", index=["Date5.1", "Code"], columns=["Specie"], fill_value=0
    )
    comb_df.reset_index(level=["Date5.1", "Code"], inplace=True)
    comb_df = pd.merge(comb_df, env_df, how="inner", on="Code")
    comb_df = comb_df.groupby(["Date5.1"], as_index=False).mean()

    scaler = StandardScaler()
    comb_df[sig_species] = scaler.fit_transform(comb_df[sig_species])
    comb_df.insert(loc=0, column="Time", value=range(len(comb_df)))

    comb_df.drop("Code", axis=1, inplace=True)
    comb_df.drop("Date5.1", axis=1, inplace=True)
    return comb_df

In [7]:
comb_df = combined_df(env, spec)

/Users/FilipW/Code_M4R/m4r_venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
comb_df

,Time,Argyrosomus.regius,Chelon.labrosus,Crangon.crangon,Cyprinus.carpio,Dicentrarchus.punctatus,Engraulis.encrasicolus,Liza.aurata,Liza.ramada,Mesopodopsis.slabberi,...,Palaemon.longirostris,Palaemon.macrodactylus,Pomatoschistus,Rhopalophthalmus.tartessicus,Sardina.pilchardus,TurbidityNTU,Temp,Si,V30,P30
0,0,-0.443243,-0.216284,-0.561308,0.007304,2.083293,-0.340024,-0.161340,-0.147774,-0.374626,...,0.096388,-0.601989,-0.645283,-0.801920,-0.368464,61.367810,22.560000,4.850000,105.032938,84.380000
1,1,-0.422784,-0.216284,-0.479096,3.131701,-0.146896,0.335733,-0.162606,-0.150417,1.126706,...,1.335908,-0.601989,-0.064054,1.433153,-0.369649,50.644014,22.873684,6.947368,104.533405,25.915789
2,2,-0.423584,-0.216284,-0.229029,0.043925,-0.260189,-0.440134,-0.162606,-0.150399,-0.272259,...,2.624835,-0.601989,-0.471475,-0.699191,-0.375247,59.572451,26.685000,9.850000,91.254989,0.000000
3,3,-0.379800,-0.216284,-0.390394,-0.077114,-0.269801,-0.282022,-0.162606,-0.150413,-0.338004,...,1.403494,-0.601989,-0.355766,-0.731114,-0.375247,62.552895,24.850000,6.700000,95.238288,1.000000
4,4,-0.430914,-0.216284,-0.425493,-0.209091,-0.234539,-0.091962,-0.162606,-0.150256,-0.523205,...,1.682288,-0.601989,-0.494152,-0.691745,-0.375247,85.722440,22.940000,5.450000,85.135622,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,146,2.029630,-0.031241,0.710307,-0.265026,-0.271428,1.541368,-0.162606,-0.150443,2.334547,...,2.959984,2.508875,0.171476,3.798488,-0.374212,101.514873,25.481818,16.363636,76.545577,0.000000
147,147,0.502336,-0.216284,-0.118186,-0.265026,-0.277415,-0.023243,-0.162606,-0.150400,-0.008880,...,1.286793,0.112240,0.270640,2.498580,-0.375247,111.895806,25.814583,13.750000,83.565979,0.000000
148,148,0.393634,-0.216284,-0.378828,-0.265026,-0.276098,-0.199188,-0.162606,-0.147949,-0.422833,...,0.699077,0.165517,-0.219609,-0.031298,-0.375247,218.711010,24.354545,13.090909,71.500092,5.000000
149,149,1.895817,-0.216284,-0.354748,-0.265026,-0.276482,0.867604,-0.162606,-0.150367,-0.000991,...,1.222556,0.490914,0.801131,-0.113664,-0.219517,117.991922,22.908333,18.916667,38.186194,28.100000


In [9]:
comb_5_flood = combined_df_ST(env, spec, site=5, tide_list=(1, 3))

/Users/FilipW/Code_M4R/m4r_venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [10]:
comb_5_flood

,Time,Argyrosomus.regius,Chelon.labrosus,Crangon.crangon,Cyprinus.carpio,Dicentrarchus.punctatus,Engraulis.encrasicolus,Liza.aurata,Liza.ramada,Mesopodopsis.slabberi,...,Palaemon.longirostris,Palaemon.macrodactylus,Pomatoschistus,Rhopalophthalmus.tartessicus,Sardina.pilchardus,TurbidityNTU,Temp,Si,V30,P30
0,0,-0.331485,-0.166065,-0.424686,-0.177056,-0.089017,-0.161639,-0.290477,-0.20757,0.417179,...,-0.322431,-0.295066,-0.817360,-0.433733,-0.223552,42.497271,22.00,19.5,117.483264,98.7
1,1,-0.247483,-0.166065,-0.333683,2.084725,-0.085204,3.981483,-0.290477,-0.20757,1.490922,...,4.336181,-0.295066,1.348952,6.839662,-0.284724,17.734822,23.50,15.0,97.384464,10.4
2,2,-0.274055,-0.166065,-0.035506,-0.154369,-0.210871,-0.052464,-0.290477,-0.20757,-0.270620,...,7.999368,-0.295066,-0.303976,-0.354812,-0.317805,57.066615,25.95,17.0,91.735200,0.0
3,3,-0.061523,-0.166065,-0.065487,-0.194974,-0.217834,0.265781,-0.290477,-0.20757,-0.252523,...,4.052698,-0.295066,0.525220,-0.441257,-0.317805,36.493061,25.65,11.0,94.817952,1.0
4,4,-0.298071,-0.166065,-0.172799,-0.194974,-0.128242,-0.183396,-0.290477,-0.20757,-0.336583,...,1.064526,-0.295066,0.161782,-0.527491,-0.317805,48.191749,24.30,8.0,93.952224,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,141,4.690781,-0.166065,0.008010,-0.194974,-0.232026,0.471442,-0.290477,-0.20757,0.824185,...,3.952686,-0.210722,0.882294,3.086351,-0.315984,49.805360,25.20,17.0,78.134242,0.0
142,142,1.242815,-0.166065,-0.169745,-0.194974,-0.232026,0.193812,-0.290477,-0.20757,-0.035341,...,1.273247,-0.250110,0.533096,0.967218,-0.317805,27.416494,26.00,16.5,83.521800,0.0
143,143,-0.011072,-0.166065,-0.313735,-0.194974,-0.232026,-0.235473,-0.290477,-0.20757,-0.289415,...,-0.180079,0.089192,-0.335314,-0.132222,-0.317805,63.218510,24.75,15.5,72.501005,5.0
144,144,0.039194,-0.166065,-0.184380,-0.194974,-0.232026,1.899141,-0.290477,-0.20757,-0.045228,...,1.979227,1.466926,0.799958,0.101473,0.527342,16.527814,23.50,18.5,44.004859,28.1


In [11]:
comb_5_ebb = combined_df_ST(env, spec, site=5, tide_list=(2, 4))

/Users/FilipW/Code_M4R/m4r_venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
comb_5_ebb

,Time,Argyrosomus.regius,Chelon.labrosus,Crangon.crangon,Cyprinus.carpio,Dicentrarchus.punctatus,Engraulis.encrasicolus,Liza.aurata,Liza.ramada,Mesopodopsis.slabberi,...,Palaemon.longirostris,Palaemon.macrodactylus,Pomatoschistus,Rhopalophthalmus.tartessicus,Sardina.pilchardus,TurbidityNTU,Temp,Si,V30,P30
0,0,-0.393906,-0.109029,-0.232098,0.300952,2.314745,-0.482442,-0.200292,-0.247383,-0.342401,...,0.665321,-0.490783,-0.256994,-0.567542,-0.342383,40.554918,23.350000,5.5,119.117952,98.7
1,1,-0.421367,-0.109029,-0.578144,-0.142269,0.071589,0.734679,-0.235589,-0.247383,-0.495443,...,-0.408696,-0.490783,-0.506069,-0.567633,-0.342890,18.541628,21.800000,32.0,93.579840,4.0
2,2,-0.421367,-0.109029,-0.567124,-0.142269,-0.280004,-0.194120,-0.235589,-0.247383,-0.386718,...,0.103617,-0.490783,-0.509655,-0.368190,-0.342890,17.936523,25.150000,33.0,91.735200,0.0
3,3,-0.421367,-0.109029,-0.558011,-0.142269,-0.281622,0.459204,-0.235589,-0.247383,-0.198626,...,0.099677,-0.490783,-0.508982,-0.393891,-0.342890,13.700792,24.350000,30.5,94.817952,1.0
4,4,-0.403804,-0.109029,-0.445641,-0.065144,-0.182222,1.272782,-0.235589,-0.240891,-0.085719,...,1.642966,-0.490783,-0.460545,-0.251853,-0.342890,17.734822,24.000000,26.0,93.952224,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,137,0.076911,-0.109029,-0.276779,-0.142269,-0.284034,0.411707,-0.235589,-0.247383,-0.006466,...,1.498612,0.348130,-0.438267,0.973789,-0.342137,9.165710,24.700000,31.5,78.134242,0.0
138,138,-0.116876,-0.109029,0.662740,-0.142269,-0.284034,0.055270,-0.235589,-0.247383,-0.313973,...,1.327001,-0.287450,0.298853,0.181671,-0.342890,29.332658,25.171667,26.0,83.521800,0.0
139,139,-0.421367,-0.109029,-0.213025,-0.142269,-0.276712,-0.146093,-0.235589,-0.210002,-0.354510,...,0.252428,0.180380,0.092854,-0.180906,-0.342890,88.935451,25.000000,24.5,71.568922,5.0
140,140,-0.176938,-0.109029,-0.407326,-0.142269,-0.284034,-0.072409,-0.235589,-0.247383,-0.447362,...,0.741338,-0.362730,-0.257925,-0.518664,-0.289501,3.921470,23.100000,33.0,44.004859,28.1


In [13]:
comb_3_ebb = combined_df_ST(env, spec, site=3, tide_list=(2, 4))

/Users/FilipW/Code_M4R/m4r_venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [14]:
comb_3_ebb

,Time,Argyrosomus.regius,Chelon.labrosus,Crangon.crangon,Cyprinus.carpio,Dicentrarchus.punctatus,Engraulis.encrasicolus,Liza.aurata,Liza.ramada,Mesopodopsis.slabberi,...,Palaemon.longirostris,Palaemon.macrodactylus,Pomatoschistus,Rhopalophthalmus.tartessicus,Sardina.pilchardus,TurbidityNTU,Temp,Si,V30,P30
0,0,-0.282675,-0.339647,-0.276263,0.543405,2.448186,-0.406396,-0.263237,-0.363416,-0.323335,...,-0.106921,-0.446476,-0.363330,-0.189519,-0.199385,63.577031,22.20,3.5,102.584448,84.7
1,1,-0.282675,-0.339647,-0.231352,10.231648,-0.228357,0.325929,-0.263237,-0.514999,1.693129,...,0.233281,-0.446476,-0.224470,-0.189519,-0.199385,46.376435,21.50,5.5,104.377248,31.0
2,2,-0.282675,-0.339647,-0.076058,0.323635,-0.266048,-0.349348,-0.263237,-0.514999,0.435547,...,0.349030,-0.446476,-0.079867,-0.189519,-0.199385,70.580615,25.80,8.0,91.246608,0.0
3,3,-0.282675,-0.339647,-0.254740,0.404567,-0.264754,-0.447804,-0.263237,-0.514999,-0.153651,...,0.437479,-0.446476,-0.089588,-0.189519,-0.199385,63.924465,24.25,3.5,95.768784,1.0
4,4,-0.282675,-0.339647,-0.235013,-0.075697,-0.266040,-0.423118,-0.263237,-0.514999,-0.310317,...,1.365915,-0.446476,-0.230571,-0.189519,-0.199385,123.023005,22.65,3.0,82.130544,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,133,-0.020401,-0.339647,-0.017149,-0.206598,-0.252962,2.308335,-0.263237,-0.514999,0.546130,...,0.017433,0.350512,-0.350630,3.785441,-0.199385,90.549063,25.40,11.5,74.823005,0.0
134,134,-0.170013,-0.339647,-0.250669,-0.206598,-0.266998,-0.168617,-0.263237,-0.496332,-0.100041,...,0.091239,0.003355,-0.388408,-0.189519,-0.199385,175.566246,25.85,8.5,83.704968,0.0
135,135,-0.282675,-0.339647,-0.254080,-0.206598,-0.267899,-0.425476,-0.263237,0.208441,-0.350882,...,0.134887,0.497021,-0.380483,-0.189519,-0.199385,657.389318,23.50,7.0,70.601674,5.0
136,136,8.180659,-0.339647,-0.184660,-0.206598,-0.264170,1.153999,-0.263237,-0.514999,0.019713,...,0.363682,0.631109,1.964487,-0.027732,-0.170606,137.343812,21.85,19.0,31.248590,28.1


In [15]:
comb_3_flood = combined_df_ST(env, spec, site=3, tide_list=(1, 3))

/Users/FilipW/Code_M4R/m4r_venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [16]:
comb_3_flood

,Time,Argyrosomus.regius,Chelon.labrosus,Crangon.crangon,Cyprinus.carpio,Dicentrarchus.punctatus,Engraulis.encrasicolus,Liza.aurata,Liza.ramada,Mesopodopsis.slabberi,...,Palaemon.longirostris,Palaemon.macrodactylus,Pomatoschistus,Rhopalophthalmus.tartessicus,Sardina.pilchardus,TurbidityNTU,Temp,Si,V30,P30
0,0,-0.296370,-0.380016,-0.263487,-0.017652,6.025926,-0.357986,-0.413412,-0.386649,-0.332958,...,-0.212441,-0.462002,-0.242386,-0.274464,-0.203045,28.408333,22.5000,2.5,102.584448,84.7
1,1,-0.296370,-0.380016,-0.230804,5.349489,0.154444,-0.161931,-0.413412,-0.403791,0.012387,...,0.966800,-0.462002,-0.191142,-0.274464,-0.203045,41.939002,23.2500,2.5,104.377248,31.0
2,2,-0.296370,-0.380016,-0.262093,0.066285,-0.297120,-0.344286,-0.413412,-0.398551,-0.028632,...,0.019161,-0.462002,-0.237140,-0.274464,-0.203045,58.075122,27.4500,4.0,91.246608,0.0
3,3,-0.296370,-0.380016,-0.261617,0.569647,-0.297551,-0.362110,-0.413412,-0.403791,-0.316483,...,1.350852,-0.462002,-0.182533,-0.274464,-0.203045,82.077601,24.6500,4.0,95.768784,1.0
4,4,-0.296370,-0.380016,-0.263487,-0.044862,-0.324893,-0.359652,-0.413412,-0.401100,-0.334693,...,2.345258,-0.462002,-0.231637,-0.274464,-0.203045,139.159125,22.6000,1.5,82.130544,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,134,-0.169699,2.771311,4.464289,-0.259091,-0.219347,7.416862,-0.413412,-0.403791,8.296794,...,6.385644,7.255071,-0.139524,2.208371,-0.203045,272.382966,26.1000,7.0,74.823005,0.0
135,135,-0.202351,-0.380016,-0.082306,-0.259091,-0.345142,-0.133530,-0.413412,-0.403791,0.065435,...,2.117776,0.705993,-0.239531,-0.176903,-0.203045,276.719548,26.0325,4.0,83.704968,0.0
136,136,-0.137287,-0.380016,-0.253281,-0.259091,-0.345142,-0.205927,-0.413412,-0.320966,-0.138478,...,0.968769,0.113795,-0.216442,0.026766,-0.203045,419.337802,23.7500,3.5,72.641232,5.0
137,137,-0.220466,-0.380016,-0.254375,-0.259091,-0.309770,-0.036506,-0.413412,-0.388687,-0.170573,...,0.292318,0.089550,-0.211767,2.847502,-0.203045,386.003234,23.0000,8.0,31.248590,28.1


In [17]:
mae_dict_combs = mae_dictionary(comb_df, max_dim=24)
mae_dict_5_flood = mae_dictionary(comb_5_flood, max_dim=24)
mae_dict_5_ebb = mae_dictionary(comb_5_ebb, max_dim=24)
mae_dict_3_flood = mae_dictionary(comb_3_flood, max_dim=24)
mae_dict_3_ebb = mae_dictionary(comb_3_ebb, max_dim=24)

In [18]:
#_, surrogates_comb, _ = generate_all_surrogates(comb_df, mae_dict = mae_dict_combs, n_surrogates=100, obs_per_year=12)
_, surrogates_comb_5_flood, _ = generate_all_surrogates(comb_5_flood, mae_dict = mae_dict_5_flood, n_surrogates=100, obs_per_year=12)
_, surrogates_comb_5_ebb, _ = generate_all_surrogates(comb_5_ebb, mae_dict = mae_dict_5_ebb, n_surrogates=100, obs_per_year=12)
_, surrogates_comb_3_flood, _ = generate_all_surrogates(comb_3_flood, mae_dict = mae_dict_3_flood, n_surrogates=100, obs_per_year=12)
_, surrogates_comb_3_ebb, _ = generate_all_surrogates(comb_3_ebb, mae_dict = mae_dict_3_ebb, n_surrogates=100, obs_per_year=12)

E = 7
Threshold = 9
E = 24
Threshold = 17
E = 3
Threshold = 4
E = 17
Threshold = 12.5
E = 3
Threshold = 12.5
E = 18
Threshold = 4
E = 11
Threshold = 16
E = 2
Threshold = 12.5
E = 2
Threshold = 12.5
E = 20
Threshold = 9
E = 19
Threshold = 4
E = 3
Threshold = 12.5
E = 4
Threshold = 4
E = 2
Threshold = 12.5
E = 12
Threshold = 17
E = 24
Threshold = 9
E = 10
Threshold = 4
E = 3
Threshold = 4
E = 4
Threshold = 4
E = 3
Threshold = 12.5
E = 11
Threshold = 10
E = 11
Threshold = 12.5
E = 24
Threshold = 9
E = 2
Threshold = 12.5
E = 20
Threshold = 10
E = 11
Threshold = 4
E = 8
Threshold = 12.5
E = 12
Threshold = 9
E = 13
Threshold = 8
E = 23
Threshold = 18
E = 19
Threshold = 17
E = 20
Threshold = 12.5
E = 24
Threshold = 10
E = 10
Threshold = 4
E = 24
Threshold = 12.5
E = 22
Threshold = 9
E = 3
Threshold = 4
E = 15
Threshold = 4
E = 20
Threshold = 20
E = 8
Threshold = 4
E = 20
Threshold = 4
E = 9
Threshold = 12.5
E = 21
Threshold = 17
E = 24
Threshold = 12
E = 14
Threshold = 12.5
E = 13
Threshold =

In [19]:
surrogates_comb_3_ebb.shape

(20, 138, 100)

In [20]:
names = comb_df.columns.values.tolist()
names.remove('Time')
names

['Argyrosomus.regius',
 'Chelon.labrosus',
 'Crangon.crangon',
 'Cyprinus.carpio',
 'Dicentrarchus.punctatus',
 'Engraulis.encrasicolus',
 'Liza.aurata',
 'Liza.ramada',
 'Mesopodopsis.slabberi',
 'Neomysis.integer',
 'Palaemon.longirostris',
 'Palaemon.macrodactylus',
 'Pomatoschistus',
 'Rhopalophthalmus.tartessicus',
 'Sardina.pilchardus',
 'TurbidityNTU',
 'Temp',
 'Si',
 'V30',
 'P30']

In [21]:
len(names)

20

In [26]:
# for i, name in enumerate(names):
#     np.save(name + '_comb5ebb_surr', surrogates_comb_5_ebb[i,:,:].T)

In [27]:
# make arrays of data
from itertools import combinations
combs = list(combinations(names,2))
combs

[('Argyrosomus.regius', 'Chelon.labrosus'),
 ('Argyrosomus.regius', 'Crangon.crangon'),
 ('Argyrosomus.regius', 'Cyprinus.carpio'),
 ('Argyrosomus.regius', 'Dicentrarchus.punctatus'),
 ('Argyrosomus.regius', 'Engraulis.encrasicolus'),
 ('Argyrosomus.regius', 'Liza.aurata'),
 ('Argyrosomus.regius', 'Liza.ramada'),
 ('Argyrosomus.regius', 'Mesopodopsis.slabberi'),
 ('Argyrosomus.regius', 'Neomysis.integer'),
 ('Argyrosomus.regius', 'Palaemon.longirostris'),
 ('Argyrosomus.regius', 'Palaemon.macrodactylus'),
 ('Argyrosomus.regius', 'Pomatoschistus'),
 ('Argyrosomus.regius', 'Rhopalophthalmus.tartessicus'),
 ('Argyrosomus.regius', 'Sardina.pilchardus'),
 ('Argyrosomus.regius', 'TurbidityNTU'),
 ('Argyrosomus.regius', 'Temp'),
 ('Argyrosomus.regius', 'Si'),
 ('Argyrosomus.regius', 'V30'),
 ('Argyrosomus.regius', 'P30'),
 ('Chelon.labrosus', 'Crangon.crangon'),
 ('Chelon.labrosus', 'Cyprinus.carpio'),
 ('Chelon.labrosus', 'Dicentrarchus.punctatus'),
 ('Chelon.labrosus', 'Engraulis.encrasicol

In [28]:
len(combs)

190

In [29]:
np.savetxt('combs_env_guadalquivir.csv', combs, delimiter = ',', fmt ='% s')

In [35]:
# for name in names:
#     temp = np.array(comb_3_flood[name], dtype = float)
#     np.save(name, temp)